# V1

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [4]:
tf.test.is_gpu_available()

True

In [5]:
!pip install contractions

     |████████████████████████████████| 245kB 8.8MB/s 
     |████████████████████████████████| 317kB 44.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81706 sha256=d00e7e3716d48bc9013c130732485bd368cdaf6e547468b744233e7f80246f49
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [0]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import re

In [0]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def expand_contractions(text):
    return contractions.fix(text)



def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text


def pre_process_document(document):
    
    # strip HTML
    document = strip_html_tags(document)
    
    # lower case
    document = document.lower()
    
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    
    # remove accented characters
    document = remove_accented_chars(document)
    
    # expand contractions    
    document = expand_contractions(document)
               
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)  
        
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


pre_process_corpus = np.vectorize(pre_process_document)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
local_data_path = '/content/drive/My Drive/ColabNotebooks/winereview/' # change as needed
reviews = pd.read_csv(local_data_path+"winemag-data-130k-v2.csv")
def point_assign(x):
  if x <= 85:
    return 0
  elif x <= 90:
    return 1
  elif x <=95:
    return 2
  else:
    return 3
reviews["points_cat"] = reviews["points"].apply(point_assign)

In [0]:
pre_reviews = reviews['description'].values

In [0]:
post_reviews = pre_process_corpus(pre_reviews)

In [12]:
post_reviews[14343]

'this rhonestyle blend is harsh and rugged with unripe mintiness but it has enough cherry berry and mocha flavors to pass'

In [0]:
X = post_reviews
y = reviews['points_cat'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [14]:
print(X[9779])
print(y[9779])
print(len(X_train))

since we last tried this wine catena has dialed back the new oak and pumped up the natural acidity and the result is pleasantly akin to fine burgundy now the emphasis is on refined dijonstyle apple and peach flavors with great mouthfeel drink now
2
64985


In [0]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': X}, y,
    batch_size=32, num_epochs=None, shuffle=True)

In [0]:
# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': X_train}, y_train, shuffle=False)

In [0]:
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': X_test}, y_test, shuffle=False)

In [0]:
embedding_feature = hub.text_embedding_column(
    key='sentence', 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1", #https://tfhub.dev/google/nnlm-en-dim128/2
    trainable=True) # change to True

In [19]:
dnn = tf.estimator.DNNClassifier(
          hidden_units=[256, 64],
          feature_columns=[embedding_feature],
          n_classes=4,
          activation_fn=tf.nn.relu,
          dropout=0.1,
          optimizer=tf.train.AdamOptimizer(learning_rate=0.001))

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpf4tmc94o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0d10b2af60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpf4tmc94o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0d10b2af60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [20]:
tf.logging.set_verbosity(tf.logging.ERROR)
import time

TOTAL_STEPS = 10000
STEP_SIZE = 1000
for step in range(0, TOTAL_STEPS+1, STEP_SIZE):
    print()
    print('-'*100)
    print('Training for step =', step)
    start_time = time.time()
    dnn.train(input_fn=train_input_fn, steps=STEP_SIZE)
    elapsed_time = time.time() - start_time
    print('Train Time (s):', elapsed_time)
    print('Eval Metrics (Train):', dnn.evaluate(input_fn=predict_train_input_fn))
    print('Eval Metrics (Testing):', dnn.evaluate(input_fn=predict_test_input_fn))


----------------------------------------------------------------------------------------------------
Training for step = 0
Train Time (s): 1136.2323458194733
Eval Metrics (Train): {'accuracy': 0.73830885, 'average_loss': 0.6091275, 'loss': 77.92156, 'global_step': 1000}
Eval Metrics (Testing): {'accuracy': 0.74128276, 'average_loss': 0.6008085, 'loss': 76.85854, 'global_step': 1000}

----------------------------------------------------------------------------------------------------
Training for step = 1000
Train Time (s): 1125.5958614349365
Eval Metrics (Train): {'accuracy': 0.7387243, 'average_loss': 0.5961304, 'loss': 76.25892, 'global_step': 2000}
Eval Metrics (Testing): {'accuracy': 0.7420214, 'average_loss': 0.5901038, 'loss': 75.48914, 'global_step': 2000}

----------------------------------------------------------------------------------------------------
Training for step = 2000
Train Time (s): 1121.1485755443573
Eval Metrics (Train): {'accuracy': 0.76579213, 'average_loss': 

In [0]:
def get_predictions(estimator, input_fn):
    return [x["class_ids"][0] for x in estimator.predict(input_fn=input_fn)]

In [0]:
y_test_predict = get_predictions(estimator=dnn, input_fn=predict_test_input_fn)

In [23]:
len(y_test_predict)

64986

In [0]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

In [25]:
confusion_matrix(y_test, y_test_predict)

array([[ 7598,  3320,    14,     0],
       [ 1569, 32780,  2954,     0],
       [   15,  3599, 12636,    59],
       [    1,     9,   222,   210]])

In [26]:
precision_recall_fscore_support(y_test, y_test_predict, average='weighted')

(0.8187367728930917, 0.8190071707752439, 0.8171293848006761, None)

In [27]:
type(dnn)

tensorflow_estimator.python.estimator.canned.dnn.DNNClassifier

In [28]:
dnn.model_dir

'/tmp/tmpf4tmc94o'

In [29]:
dnn.get_variable_names()

['dnn/head/beta1_power',
 'dnn/head/beta2_power',
 'dnn/hiddenlayer_0/bias',
 'dnn/hiddenlayer_0/bias/t_0/Adam',
 'dnn/hiddenlayer_0/bias/t_0/Adam_1',
 'dnn/hiddenlayer_0/kernel',
 'dnn/hiddenlayer_0/kernel/t_0/Adam',
 'dnn/hiddenlayer_0/kernel/t_0/Adam_1',
 'dnn/hiddenlayer_1/bias',
 'dnn/hiddenlayer_1/bias/t_0/Adam',
 'dnn/hiddenlayer_1/bias/t_0/Adam_1',
 'dnn/hiddenlayer_1/kernel',
 'dnn/hiddenlayer_1/kernel/t_0/Adam',
 'dnn/hiddenlayer_1/kernel/t_0/Adam_1',
 'dnn/input_from_feature_columns/input_layer/sentence_hub_module_embedding/module/embeddings',
 'dnn/input_from_feature_columns/input_layer/sentence_hub_module_embedding/module/embeddings/t_0/Adam',
 'dnn/input_from_feature_columns/input_layer/sentence_hub_module_embedding/module/embeddings/t_0/Adam_1',
 'dnn/logits/bias',
 'dnn/logits/bias/t_0/Adam',
 'dnn/logits/bias/t_0/Adam_1',
 'dnn/logits/kernel',
 'dnn/logits/kernel/t_0/Adam',
 'dnn/logits/kernel/t_0/Adam_1',
 'global_step']